In [962]:
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from boruta import BorutaPy
from collections import Counter
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from knnmv_master.impute import KNNMVImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [1431]:
train = pd.read_csv('Interm/train.csv')
test = pd.read_csv('Interm/test.csv')

# train.fillna('NaN', inplace=True)
# test.fillna('NaN', inplace=True)

In [1432]:
train_index = train.shape[0]
combine = train.append(test, ignore_index=True, sort=False)

In [1433]:
# removed 'City_Type','Employer_Category'
list_one_hot_features = ['City_Type','Employer_Category','Category2','Category3']
target_cols = ['Target_1','Target_2']

# One hot encoding
for i in list_one_hot_features:
    temp_one_hot = pd.get_dummies(combine[[i]], columns=[i])
    combine = combine.join(temp_one_hot)

In [1401]:
train = combine.iloc[0:train_index].reset_index(drop=True)
test = combine.iloc[train_index:].reset_index(drop=True)

## Training for camp type 1 and 2 (Favorable Outcome: Getting a health score)
** HS: Health Score

In [1402]:
trainHS = train[(train['Category1'] == 'First') | (train['Category1'] == 'Second')].reset_index(drop=True)

testHS = test[(test['Category1'] == 'First') | (test['Category1'] == 'Second')].reset_index(drop=True)

In [1403]:
X = trainHS[set(trainHS.columns).difference(set(['Patient_ID','Health_Camp_ID','Category1'] \
                                                + list_one_hot_features + target_cols))]

y = trainHS['Target_1']

tempHS = testHS[['Patient_ID','Health_Camp_ID']]
testHS = testHS[set(testHS.columns).difference(set(['Patient_ID','Health_Camp_ID','Category1'] \
                                                                  + list_one_hot_features + target_cols))]


#### Train test split

In [1404]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, shuffle=True) # Shuffle = True

train_X = train_X.reset_index(drop=True); train_y = train_y.reset_index(drop=True);
test_X = test_X.reset_index(drop=True); test_y = test_y.reset_index(drop=True)

#### Feature Selection

In [1405]:
import os
cwd = os.getcwd()

%cd C:/Users/Lakhan/Desktop/Wyng/Demand Forecasting
from FeatureSelectionPipe import FeatureSelector

os.chdir(cwd)

C:\Users\Lakhan\Desktop\Wyng\Demand Forecasting


In [1406]:
# Define steps
step1 = {'Constant Features': {'frac_constant_values': 0.95}}

step2 = {'Correlated Features': {'correlation_threshold': 0.95}}

step3 = {'Multicorrelated Features': {'acceptable_vif_threshold': 50}}

# # For RFECV Features, DecisionTreeRegressor can also be used as an estimator
# estimator = RandomForestClassifier(random_state = 42, n_estimators = 20) 

# step5 = {'RFECV Features': {'estimator': estimator,
#                     'cv': TimeSeriesSplit(n_splits=3), 
#                     'step': 1,
#                     'scoring': 'accuracy', 
#                     'verbose': 50}}

steps = [step1, step2, step3]

# Initialize FeatureSelector()
fs = FeatureSelector()

# Apply feature selection methods
fs.fit(train_X, train_y, steps)

fs.selected_features

selected_features = list(set(fs.selected_features + fs.critical_features))

Removing Constant Features
['Var2', 'Employer_Category_Food', 'Employer_Category_Retail', 'Var3', 'Employer_Category_Technology', 'Category3_2', 'Category2_G', 'City_Type_A', 'City_Type_F', 'Employer_Category_Broadcasting', 'Employer_Category_Software Industry', 'Category2_C', 'Employer_Category_Consulting', 'Employer_Category_Telecom', 'Category3_1', 'Employer_Category_Real Estate', 'Var4', 'Employer_Category_Education', 'Category2_B', 'Employer_Category_Transport', 'Employer_Category_Others', 'Employer_Category_BFSI', 'Employer_Category_Manufacturing', 'City_Type_I', 'Employer_Category_Health']

Removing Correlated Features
[]

Removing Multicorrelated Features


c:\users\lakhan\appdata\local\programs\python\python37\lib\site-packages\statsmodels\stats\outliers_influence.py:181: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


['Interaction_to_Camp_Start_Date', 'Registration_to_Camp_End_Date']

Done selecting features


['City_Type_C',
 'Age',
 'Education_Score',
 'Category2_F',
 'City_Type_E',
 'Var1',
 'City_Type_D',
 'Twitter_Shared',
 'Online_Follower',
 'Income',
 'Facebook_Shared',
 'LinkedIn_Shared',
 'City_Type_H',
 'Interaction_to_Registration',
 'Registration_to_Camp_Start_Date',
 'Interaction_to_Camp_End_Date',
 'Category2_E',
 'City_Type_B',
 'Category2_D',
 'City_Type_NaN',
 'Employer_Category_NaN',
 'Category2_A',
 'Var5',
 'City_Type_G']

In [1407]:
train_X_selected = fs.transform(train_X)
test_X_selected = fs.transform(test_X)

testHS = fs.transform(testHS)

In [1408]:
# applying smote
# sm = SMOTE(sampling_strategy='auto',random_state=42)
sm = BorderlineSMOTE(sampling_strategy='auto',random_state=42)

X_res, y_res = sm.fit_resample(train_X_selected, train_y)
print('Original dataset shape %s' % Counter(train_y))
print('Resampled dataset shape %s' % Counter(y_res))

# train_X_selected, train_y
# X_res, y_res

Original dataset shape Counter({0.0: 40659, 1.0: 11079})
Resampled dataset shape Counter({0.0: 40659, 1.0: 40659})


In [1409]:
import xgboost as xgb
from catboost import CatBoostClassifier

# estimator = CatBoostClassifier(verbose=False, boosting_type='Plain')

params = {'colsample_bytree': 1, 'eta': 0.1, 'max_depth': 7, 'n_estimators': 100, 
                'objective': 'binary:logistic', 'subsample': 0.8,}

# c = Counter(train_y)
#  'scale_pos_weight': c[0]/c[1]
estimator = xgb.XGBClassifier()

# # XGBClassifier
# # early_stopping_rounds: The number of rounds without improvements after which we should stop
# param_grid = { 'objective': ['binary:logistic'],
#               'max_depth': [5,6,7], # 'min_child_weight': [5,6,7,8],
#               'subsample': [0.8,0.9,1], 'colsample_bytree': [0.8,0.9,1],
# # eta parameter: https://www.kaggle.com/c/santander-customer-satisfaction/discussion/20208
#               'eta': [0.1], 'n_estimators': [25, 50, 100]
#          }


# # Initialize GridSearch object with 5-fold cross validation
# # error_score = 0 silences any exceptions for incorrect param combinations
# gscv = GridSearchCV(estimator, param_grid, cv = 3,  n_jobs= -1, verbose = 1, scoring = 'roc_auc', error_score=0)

# # Fit gscv
# gscv = gscv.fit(train_X_selected, train_y)

# # Get best parameters and score
# best_params = gscv.best_params_
# best_score = gscv.best_score_
        
# # Update classifier parameters
# estimator = estimator.set_params(**best_params)

# Fit classifier
estimator = estimator.fit(train_X_selected, train_y)

# Make predictions
train_y_pred = estimator.predict(train_X_selected)
test_y_pred = estimator.predict(test_X_selected)

# Measure performance
roc_auc_score_train = roc_auc_score(train_y, train_y_pred)
roc_auc_score_test = roc_auc_score(test_y, test_y_pred)

# Message to user
print(f'The roc_auc_score of the classifier on the train set was: {roc_auc_score_train*100}')
print(f'The roc_auc_score of the classifier on the test set was: {roc_auc_score_test*100}')

The roc_auc_score of the classifier on the train set was: 67.72169936792827
The roc_auc_score of the classifier on the test set was: 67.20108665872793


In [1410]:
tempHS['Outcome'] = np.round(estimator.predict_proba(testHS)[:,1], 3)

## Training for camp type 3 (Favorable Outcome: Visiting a stall)
** VS: Visiting a stall

In [1411]:
trainVS = train[train['Category1'] == 'Third'].reset_index(drop=True)
testVS = test[test['Category1'] == 'Third'].reset_index(drop=True)

In [1412]:
X = trainVS[set(trainVS.columns).difference(set(['Patient_ID','Health_Camp_ID','Category1'] \
                                                + list_one_hot_features + target_cols))]

y = trainVS['Target_2']

tempVS = testVS[['Patient_ID','Health_Camp_ID']]
testVS = testVS[set(testVS.columns).difference(set(['Patient_ID','Health_Camp_ID','Category1'] \
                                                            + list_one_hot_features+ target_cols))]


#### Train test split

In [1413]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, shuffle=True) # Shuffle = True

train_X = train_X.reset_index(drop=True); train_y = train_y.reset_index(drop=True);
test_X = test_X.reset_index(drop=True); test_y = test_y.reset_index(drop=True)

#### Feature Selection

In [1414]:
import os
cwd = os.getcwd()

%cd C:/Users/Lakhan/Desktop/Wyng/Demand Forecasting
from FeatureSelectionPipe import FeatureSelector

os.chdir(cwd)

C:\Users\Lakhan\Desktop\Wyng\Demand Forecasting


In [1415]:
# Define steps
step1 = {'Constant Features': {'frac_constant_values': 0.95}}

step2 = {'Correlated Features': {'correlation_threshold': 0.95}}

step3 = {'Multicorrelated Features': {'acceptable_vif_threshold': 50}}

# # For RFECV Features, DecisionTreeRegressor can also be used as an estimator
# estimator = RandomForestClassifier(random_state = 42, n_estimators = 20) 

# step5 = {'RFECV Features': {'estimator': estimator,
#                     'cv': TimeSeriesSplit(n_splits=3), 
#                     'step': 1,
#                     'scoring': 'accuracy', 
#                     'verbose': 50}}

steps = [step1, step2, step3]

# Initialize FeatureSelector()
fs = FeatureSelector()

# Apply feature selection methods
fs.fit(train_X, train_y, steps)

fs.selected_features

selected_features = list(set(fs.selected_features + fs.critical_features))

Removing Constant Features
['Var2', 'Employer_Category_Food', 'Employer_Category_Retail', 'Var3', 'Employer_Category_Technology', 'Category2_F', 'Category3_2', 'Category2_G', 'City_Type_F', 'Employer_Category_Broadcasting', 'Employer_Category_Software Industry', 'Category2_C', 'Twitter_Shared', 'Employer_Category_Consulting', 'Employer_Category_Telecom', 'Category3_1', 'Online_Follower', 'Employer_Category_Real Estate', 'Facebook_Shared', 'Var4', 'Employer_Category_Education', 'Category2_B', 'Employer_Category_Transport', 'Category2_E', 'Employer_Category_Others', 'Employer_Category_BFSI', 'Employer_Category_Manufacturing', 'Category2_D', 'City_Type_I', 'Category2_A', 'Employer_Category_Health']

Removing Correlated Features
['Interaction_to_Registration', 'Interaction_to_Camp_End_Date']

Removing Multicorrelated Features
['Education_Score']

Done selecting features


['City_Type_C',
 'Age',
 'Interaction_to_Camp_Start_Date',
 'City_Type_E',
 'Var1',
 'City_Type_A',
 'Registration_to_Camp_End_Date',
 'City_Type_D',
 'Income',
 'LinkedIn_Shared',
 'City_Type_H',
 'Registration_to_Camp_Start_Date',
 'City_Type_B',
 'City_Type_NaN',
 'Employer_Category_NaN',
 'Var5',
 'City_Type_G']

In [1416]:
train_X_selected = fs.transform(train_X)
test_X_selected = fs.transform(test_X)

testVS = fs.transform(testVS)

In [1417]:
# applying smote
sm = SMOTE(sampling_strategy='auto',random_state=42)
# sm = BorderlineSMOTE(sampling_strategy='auto',random_state=42)

X_res, y_res = sm.fit_resample(train_X_selected, train_y)
print('Original dataset shape %s' % Counter(train_y))
print('Resampled dataset shape %s' % Counter(y_res))

# train_X_selected, train_y
# X_res, y_res

Original dataset shape Counter({1.0: 5207, 0.0: 3009})
Resampled dataset shape Counter({1.0: 5207, 0.0: 5207})


In [1418]:
import xgboost as xgb
from catboost import CatBoostClassifier

# estimator = CatBoostClassifier(verbose=False, boosting_type="Plain")
# Initiate classifier instance
params = {'colsample_bytree': 1, 'eta': 0.1, 'max_depth': 5, 'min_child_weight': 7, 
            'n_estimators': 50, 'objective': 'binary:logistic', 'subsample': 0.9}

# estimator = Pipeline([('scaler', MinMaxScaler()), ('lr', xgb.XGBClassifier())])

estimator = xgb.XGBClassifier()
# # XGBClassifier
# # early_stopping_rounds: The number of rounds without improvements after which we should stop
# param_grid = { 'objective': ['binary:logistic'],
#               'max_depth': [3,4,5,6], 'min_child_weight': [5,6,7,8],
#               'subsample': [0.7,0.8,0.9,1], 'colsample_bytree': [0.7,0.8,0.9,1],
# # eta parameter: https://www.kaggle.com/c/santander-customer-satisfaction/discussion/20208
#               'eta': [0.1,0.05], 'n_estimators': [25, 50, 100]
#          }

# param_grid = { 'scale_pos_weight': [2,]}

# # Initialize GridSearch object with 5-fold cross validation
# # error_score = 0 silences any exceptions for incorrect param combinations
# gscv = GridSearchCV(estimator, param_grid, cv = 5,  n_jobs= -1, verbose = 1, scoring = 'roc_auc', error_score=0)

# # Fit gscv
# gscv = gscv.fit(train_X, train_y)

# # Get best parameters and score
# best_params = gscv.best_params_
# best_score = gscv.best_score_
        
# # Update classifier parameters
# estimator = estimator.set_params(**best_params)

# Fit classifier
estimator = estimator.fit(train_X_selected, train_y)

# Make predictions
train_y_pred = estimator.predict(train_X_selected)
test_y_pred = estimator.predict(test_X_selected)

# Measure performance
roc_auc_score_train = roc_auc_score(train_y, train_y_pred)
roc_auc_score_test = roc_auc_score(test_y, test_y_pred)

# Message to user
print(f'The roc_auc_score of the classifier on the train set was: {roc_auc_score_train*100}')
print(f'The roc_auc_score of the classifier on the test set was: {roc_auc_score_test*100}')

The roc_auc_score of the classifier on the train set was: 68.62410017243577
The roc_auc_score of the classifier on the test set was: 66.99737939273922


In [1419]:
tempVS['Outcome'] = np.round(estimator.predict_proba(testVS)[:,1], 3)

#### Submission

In [1420]:
result = tempHS.append(tempVS, ignore_index=True, sort=False)

result.to_csv('Submissions/new_features.csv', index=0)

In [1421]:
result

,Patient_ID,Health_Camp_ID,Outcome
0,500633,6584,0.403
1,506945,6582,0.112
2,497447,6551,0.272
3,496446,6533,0.077
4,525212,6567,0.663
5,507387,6583,0.729
6,500798,6572,0.249
7,489618,6551,0.252
8,495624,6582,0.149
9,488974,6566,0.111
